In [15]:
#importando modulos
import glob
import pandas as pd

#### Importando todos os arquivos .csv da pasta database/raw

In [64]:
# Listando todos os arquivos
csv_files = glob.glob('*.{}'.format('csv'))

# Criando lista vazia
dfs = list()

# Ler cada arquivo listado e entao adiciona a lista vaiza criada acima
for f in csv_files:
    data = pd.read_csv(f, sep = ',', encoding = 'latin1')
    dfs.append(data)   

# Transforma a lista em um dataframe
frame = pd.concat(dfs, axis=0, ignore_index=True) 

# Deleta valores duplicados
frame.drop_duplicates(inplace=True)

#### Limpando dataframe gerado

In [65]:
# Selecionando quais colunas manter no dataframe
frame = frame[['CNES', 'CODUFMUN', 'CPF_CNPJ', 'TP_LEITO', 'CODLEITO', 'QT_EXIST', 'QT_SUS', 'COMPETEN']]
frame['QT_EXIST0'] = frame['QT_EXIST']
frame['QT_EXIST'] = frame['QT_EXIST'] - frame['QT_SUS']

# Criando a coluna Mês e a coluna ano
frame['MES'] = frame['COMPETEN']%100
frame['ANO'] = frame['COMPETEN']//100

# Deletando a coluna COMPETEN, pois está não há ordenação
frame.drop(['COMPETEN'], axis=1, inplace = True)

# Adicionando o zero na frente dos valores dos meses de 1 a 9.
new_list = []
for value in frame['MES']:
    if value<10:
        new_list.append('0' + str(value))
    else: 
        new_list.append(str(value))
new_column = pd.DataFrame(new_list)
frame['MES'] = new_column

# Renomeando dataframe
df = frame

# Criando a coluna date no formato MES-ANO
df['date'] = df['ANO'].astype(str) + "-" + df['MES'].astype(str)

#### Salvando dataframe

In [66]:
# Salvando dataframe base gerado dos arquivos csv
df.to_csv('../processed/df_base.csv')  

In [67]:
df = pd.read_csv('../processed/df_base.csv')

In [68]:
df

,Unnamed: 0,CNES,CODUFMUN,CPF_CNPJ,TP_LEITO,CODLEITO,QT_EXIST,QT_SUS,QT_EXIST0,MES,ANO,date
0,0,9331603,520010,0,2,33,0,9,9,9,2021,2021-09
1,1,2335506,520013,269860000125,5,45,0,3,3,9,2021,2021-09
2,2,2335506,520013,269860000125,2,33,2,4,6,9,2021,2021-09
3,3,2335506,520013,269860000125,6,34,1,3,4,9,2021,2021-09
4,4,2335506,520013,269860000125,4,10,0,1,1,9,2021,2021-09
...,...,...,...,...,...,...,...,...,...,...,...,...
103493,103493,2383691,522205,0,1,3,0,3,3,7,2021,2021-07
103494,103494,2383691,522205,0,1,6,0,1,1,7,2021,2021-07
103495,103495,2439883,522220,0,4,43,0,2,2,7,2021,2021-07
103496,103496,2439883,522220,0,2,33,0,7,7,7,2021,2021-07


In [69]:
# Criação de dataframe agrupando pelo mes/ano dos dados
df2 = df.groupby(['date'])['QT_EXIST0'].sum().reset_index()

# Salvando dataframe
df2.to_csv('../processed/df2_base.csv')  

In [70]:
# Criação de dataframe agrupando pelo mes/ano dos dados
df2 = df.groupby(['date'])[['QT_EXIST0','QT_SUS']].sum().reset_index()

# Salvando dataframe
df2.to_csv('../processed/df2_base_sus.csv')  

Criação de pivot table agrupando dados totais

In [71]:
df = pd.read_csv('../processed/df_base.csv')

In [72]:
import numpy as np

table = pd.pivot_table(df, values='QT_EXIST0', index='ANO',
                    columns=['MES'], aggfunc=np.sum)

table

MES,1,2,3,4,5,6,7,8,9,10,11,12
ANO,,,,,,,,,,,,
2020,19351.0,19311.0,19761.0,19971.0,20028.0,20290.0,20630.0,20868.0,20962.0,20989.0,20755.0,21140.0
2021,21187.0,21592.0,22066.0,22098.0,22169.0,22323.0,22525.0,22722.0,22758.0,22833.0,22886.0,23019.0
2022,23003.0,22730.0,22444.0,22395.0,22381.0,21212.0,21303.0,21343.0,21315.0,21389.0,21225.0,NaN


In [73]:
df.fillna(0)

,Unnamed: 0,CNES,CODUFMUN,CPF_CNPJ,TP_LEITO,CODLEITO,QT_EXIST,QT_SUS,QT_EXIST0,MES,ANO,date
0,0,9331603,520010,0,2,33,0,9,9,9,2021,2021-09
1,1,2335506,520013,269860000125,5,45,0,3,3,9,2021,2021-09
2,2,2335506,520013,269860000125,2,33,2,4,6,9,2021,2021-09
3,3,2335506,520013,269860000125,6,34,1,3,4,9,2021,2021-09
4,4,2335506,520013,269860000125,4,10,0,1,1,9,2021,2021-09
...,...,...,...,...,...,...,...,...,...,...,...,...
103493,103493,2383691,522205,0,1,3,0,3,3,7,2021,2021-07
103494,103494,2383691,522205,0,1,6,0,1,1,7,2021,2021-07
103495,103495,2439883,522220,0,4,43,0,2,2,7,2021,2021-07
103496,103496,2439883,522220,0,2,33,0,7,7,7,2021,2021-07


Criando dataframe com a diferença mes a mes por cidade

In [74]:
df_pivot = df.pivot_table(index='CODUFMUN', columns='date', values='QT_EXIST0', aggfunc= 'sum')

df_pivot.fillna(0, inplace = True)

df_pivot.to_csv('../processed/df_pivot.csv')  

In [76]:
df2_pivot = df_pivot.sub(df_pivot.shift(axis=1), axis=1)
df2_pivot = df2_pivot.drop('2020-01', axis=1)
df2_pivot 

date,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,...,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11
CODUFMUN,,,,,,,,,,,,,,,,,,,,,
520010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
520013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
520017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
520025,0.0,0.0,0.0,200.0,0.0,22.0,8.0,10.0,0.0,-237.0,...,0.0,8.0,0.0,0.0,-33.0,0.0,0.0,0.0,0.0,0.0
520030,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522170,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,0.0,0.0
522185,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-4.0,0.0,0.0,0.0,0.0,0.0
522200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
df2_reveted = df2_pivot.reset_index().melt(id_vars='CODUFMUN', 
                                            value_vars=df2_pivot.columns, 
                                            var_name='date', 
                                            value_name='QT_EXIST0'
                                            )


In [78]:
df2_reveted_positive = df2_reveted.query("QT_EXIST0 > 0").groupby('date')['QT_EXIST0'].sum().reset_index()

df2_reveted_negative = df2_reveted.query("QT_EXIST0 < 0").groupby('date')['QT_EXIST0'].sum().reset_index()

df2_reveted_variation = df2_reveted.query("QT_EXIST0 ! = 0").groupby('date')['QT_EXIST0'].count().reset_index()

df2_reveted_std = df2_reveted.query("QT_EXIST0 ! = 0").groupby('date')['QT_EXIST0'].std().reset_index()



In [79]:
df2_reveted_negative.to_csv('../processed/df_reverted_sub_negative.csv')  
df2_reveted_positive.to_csv('../processed/df_reverted_sub_positive.csv')  
df2_reveted_variation.to_csv('../processed/df_reverted_sub_count.csv')  
df2_reveted_std.to_csv('../processed/df_reverted_sub_std.csv')  

In [80]:
df3_pivot = (df_pivot.sub(df_pivot.shift(axis=1), axis=1)/df_pivot)*100
df3_pivot = df3_pivot.drop('2020-01', axis=1)
df3_pivot.fillna(0, inplace = True)
df3_pivot

date,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,...,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11
CODUFMUN,,,,,,,,,,,,,,,,,,,,,
520010,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
520013,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
520017,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
520025,0.0,0.0,0.0,82.987552,0.0,8.365019,2.952030,3.558719,0.0,-538.636364,...,0.0,9.302326,0.0,0.0,-62.264151,0.0,0.0,0.0,0.0,0.0
520030,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522170,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,-4.255319,0.0,0.0,0.0,0.0,0.0
522185,0.0,0.0,0.0,0.000000,0.0,0.000000,1.449275,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,-5.555556,0.0,0.0,0.0,0.0,0.0
522200,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [82]:
df3_reveted = df3_pivot.reset_index().melt(id_vars='CODUFMUN', 
                                            value_vars=df3_pivot.columns, 
                                            var_name='date', 
                                            value_name='QT_EXIST0'
                                            )
df3_reveted_positive = df3_reveted.query("QT_EXIST0 > 0").groupby('date')['QT_EXIST0'].sum().reset_index()

df3_reveted_negative = df3_reveted.query("QT_EXIST0 < 0").groupby('date')['QT_EXIST0'].sum().reset_index()
df3_reveted_negative['QT_EXIST0'] = df3_reveted_negative['QT_EXIST0'].fillna(0)
df3_reveted_negative['QT_EXIST0'] = df3_reveted_negative['QT_EXIST0'].replace([-np.inf], 0)


In [83]:
df3_reveted_negative.to_csv('../processed/df_reverted_percentage_negative.csv')  
df3_reveted_positive.to_csv('../processed/df_reverted_percentage_positive.csv')  

### Analisando períodos de redução

In [36]:
import pandas as pd

In [59]:
df = pd.read_csv('../processed/df_base.csv')

In [84]:
df_reducao_11 = df[df.date == '2020-10'].groupby('CODUFMUN')['QT_EXIST0'].sum().reset_index()
df_reducao_12 = df[df.date == '2020-11'].groupby('CODUFMUN')['QT_EXIST0'].sum().reset_index()


df_reducao_21 = df[df.date == '2021-12'].groupby('CODUFMUN')['QT_EXIST0'].sum().reset_index()
df_reducao_22 = df[df.date == '2022-06'].groupby('CODUFMUN')['QT_EXIST0'].sum().reset_index()


df_reducao_12

,CODUFMUN,QT_EXIST0
0,520010,9
1,520013,72
2,520017,12
3,520025,44
4,520030,34
...,...,...
182,522170,27
183,522185,69
184,522200,40
185,522205,16
